In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
import os


In [2]:
A = pd.read_csv('D:/Dori/fMRI data/rs_fmri/template/labels/D.txt', sep="\t", header=None)
A.loc[-1] = ['clear label']  # adding a row
A.index = A.index + 1  # shifting index
A = A.sort_index()
A.reset_index(drop=False, inplace=True)
A.columns=['Label Id', 'Label Name']
A.head()

,Label Id,Label Name
0,0,clear label
1,1,98 voxels covering 83% of D.G compositus rostr...
2,2,95 voxels covering 14% of D.encephalon encephalon
3,3,74 voxels covering 45% of D.G proreus L
4,4,63 voxels covering 53% of D.G suprasylvius ros...


In [3]:
data = pd.read_csv('D:/Dori/fMRI data/rs_fmri/template/labels/D_vol_stats.txt', sep="\t", header=0)
data['ROI_vox_size']=data['Volume (mm^3)']/8
data['ROI_vox_size']=data['ROI_vox_size'].round(0)
data.head()

,Label Id,Label Name,Number Of Voxels,Volume (mm^3),Image mean (D),Image stdev (D),ROI_vox_size
0,0,Clear Label,12507381,195428.000,0,0,24428.0
1,1,Label 1,349908,5467.310,1,0,683.0
2,2,Label 2,15715,245.547,2,0,31.0
3,3,Label 3,19563,305.672,3,0,38.0
4,4,Label 4,81919,1279.980,4,0,160.0


In [4]:
atlas=data[['Label Id','Number Of Voxels', 'Volume (mm^3)','ROI_vox_size']].merge(A, on='Label Id')
atlas=atlas.iloc[2:,:]
atlas.reset_index(inplace=True, drop=True)
atlas[atlas['Label Name']=='LPVI_1']

,Label Id,Number Of Voxels,Volume (mm^3),ROI_vox_size,Label Name


In [5]:
os.chdir("C:/Users/dooora/Desktop/jupyter/mr/rs_rois/")
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk("C:/Users/dooora/Desktop/jupyter/mr/rs_rois"):
    f.extend(filenames)
    break
    
f

['A_network.txt',
 'B_network.txt',
 'C_network.txt',
 'D_network.txt',
 'E_network.txt',
 'F_network.txt',
 'G_network.txt',
 'H_network.txt',
 'I_network.txt',
 'J_network.txt',
 'K_network.txt',
 'L_network.txt',
 'M_network.txt']

In [9]:
df=pd.DataFrame(columns=['network','voxels','percentage','ROI'])

In [15]:
for i in f:
    with open(i,'r') as f:
        raw_corrupt = f.read()
        corrupt_split=re.split('\n| of|covering',raw_corrupt)
        vox_count=corrupt_split[0::3]
        pct=corrupt_split[1::3]
        roi=corrupt_split[2::3]
        del vox_count[-1]
        df1 = pd.DataFrame({'network':i,'voxels':vox_count, 'percentage':pct, 'ROI':roi})
        df=pd.concat([df,df1])
        
df['vox_no']=df.voxels.str[:-8]
df['network']=df.network.str[0]
df['percentage']=df.percentage.str[:-1]
df['percentage']=df['percentage'].astype('int')
df['percentage']=df['percentage']/100
df['vox_no']=df['vox_no'].astype('int')
df["ROI_str"]=df["ROI"].str[3:]
#filtering out useless ROIs
df = df[df.ROI != ' A.encephalon encephalon']
df = df[df.ROI != ' A.not-labeled']
df = df[df.ROI != ' A.nervus opticus']

ValueError: I/O operation on closed file.

df[df['network']=='A']

In [ ]:
#now retaining small_blobs as well
#df2=df[df.small_blob != 1].copy()
df2=df.merge(atlas, left_on='ROI_str', right_on='Label Name')

df4=df2[['network','vox_no']].groupby('network').agg(['count','sum'])
df4.reset_index(drop=False, inplace=True)
df4.columns=['network','count','sum']
networksize=pd.Series(df4['sum'].values,index=df4['network']).to_dict()

df2['total_netw_size'] = df2['network'].map(networksize)
df2['ROI_contr_to_netw']=df2['ROI_vox_size']/df2['total_netw_size']
df2['blob_contr_to_netw']=df2['vox_no']/df2['total_netw_size']
df2['percentage_corr']=df2['vox_no']/df2['ROI_vox_size']
df2['ROI_stem'] = df2['ROI_str'].str[:-1]
df2.head()

In [ ]:
def small_insign(x,y):
    if x<10 and y<0.1:
        return 1
    else:
        return 0

df2['small_blob']=np.vectorize(small_insign)(df2['vox_no'],df2['percentage_corr'])

In [ ]:
cleaned_df=df2[['network', 'vox_no', 'ROI_str', 'small_blob', 'Label Id', 'Number Of Voxels', 'Volume (mm^3)',
       'ROI_vox_size', 'total_netw_size', 'ROI_contr_to_netw',
       'blob_contr_to_netw', 'percentage_corr', 'ROI_stem']].copy()

In [ ]:
cols2=['ROI_str', 'vox_no','ROI_vox_size',
       'percentage_corr','blob_contr_to_netw','ROI_contr_to_netw','network','total_netw_size', 'Number Of Voxels',
 'Volume (mm^3)','small_blob','Label Id', 'ROI_stem',
 ]
cleaned_df= cleaned_df[cols2]
cleaned_df.columns=['ROI_name', 'blob_vx_within_ROI','total_ROI_vx_size',
       'cover_percentage','blob_contribution_to_network','ROI_contribution_to_network','network_from_SciRep','total_network_vx_size', 'orig_small_total_vx_in_network',
 'total_ROI_vol_mm3','small_blob_flag','Label_Id','ROI_stem']
cleaned_df.head()

In [ ]:
cleaned_df[cleaned_df['ROI_name']=='LPVI_1']

In [ ]:
from collections import Counter
#checking bilaterality
bilat_df=cleaned_df[cleaned_df['small_blob_flag']!=1].copy()
end_list=pd.DataFrame()
networks=bilat_df['network_from_SciRep'].unique()
for i in networks:
    bilat_list=[]
    df=bilat_df[bilat_df['network_from_SciRep']==i]
    ROIS=df.ROI_stem.to_list()
    cnt = Counter()
    for k in ROIS:
        cnt[k] += 1
        
    for l in cnt.keys():    
        a=cnt.get(l)
        if a>1:
            bilat_list.append(l)
        else:
            pass
    df['bilat']=df.ROI_stem.apply(lambda x: 1 if x in bilat_list else 0)
    end_list=pd.concat([end_list,df])
end_list.tail()

In [ ]:
#cnt.get('G diagonalis ')
cnt.keys()

In [ ]:
df4.columns=['network','no_of_ROIs_in_network', 'total_vx_size_of_network']

In [ ]:
end_list=end_list.sort_values(by=['blob_vx_within_ROI'], ascending=False)
exclusive=end_list.drop_duplicates(subset='ROI_name', keep='first')
exclusive.head()

In [ ]:
list_dfs= [cleaned_df,df4,end_list,exclusive]
from pandas import ExcelWriter
def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()
        
save_xls(list_dfs, "C:/Users/dooora/Desktop/jupyter/mr/dog_networkROI.xlsx")        

In [ ]:
df2['blob_contr_to_netw'].hist()

one_hot=pd.get_dummies(df2['ROI'])
df3=pd.concat([df2,one_hot], axis=1, sort=False)
one_hot.head()

In [ ]:
# Customize matplotlib
plt.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
    }
)
plt.scatter(df4['count'], df4['sum'])
plt.xlabel('count')
plt.ylabel('sum')

In [ ]:
df.ROI.value_counts()

#this is network level info
df5=df.drop_duplicates(subset='ROI', keep='first').copy()
df5['ROI_size']=1/df5['percentage']*df5['vox_no']
df5['ROI_size']=df5['ROI_size'].round(0)
df5['total_netw_size'] = df5['network'].map(networksize)
df5['ROI_contr_to_netw']=df5['ROI_size']/df5['total_netw_size']
df5['blob_contr_to_netw']=df5['vox_no']/df5['total_netw_size']
df5.sort_values(by=['blob_contr_to_netw'], inplace=True, ascending=False)
df5.head(10)

In [ ]:
df5[df5['ROI'] == ' A.G splenialis L']

In [ ]:
df['ROI'][0]

In [ ]:
#to do drop rows with pőercentage below 10%
#if smaller than 5 voxel than do this, above that it may not be a good idea

In [ ]:
df5['percentage'].describe()

In [ ]:
df['percentage']

In [ ]:
plt.hist(df['vox_no'], bins=(0,1,2,3,4,5,6,7,8,10,20,50,100))